In [1]:
import geopandas as gpd
import pandas as pd
import psrcelmerpy

In [2]:
# Load shapefile where external blocks from adjacent counties have been assigned an external station
block_ext_gdf = gpd.read_file(r'R:\e2projects_two\2023_base_year\externals\adjacent_county_blocks2020.shp')

In [3]:
# We then need to associate each block group with a TAZ inside PSRC region
# Then we will have the OTAZ and DTAZ

eg_conn = psrcelmerpy.ElmerGeoConn()
gdf_taz = eg_conn.read_geolayer('taz2010')

# convert blocks to points
gdf_blocks = eg_conn.read_geolayer('BLOCK2020')
gdf_blocks_pt = gdf_blocks.copy()
gdf_blocks_pt['geometry'] = gdf_blocks_pt['geometry'].centroid
block_taz_gdf = gpd.sjoin(gdf_blocks_pt, gdf_taz)
block_taz_gdf['geoid20'] =block_taz_gdf['geoid20'].astype('str')

C:\Users\Modeller\AppData\Local\anaconda3\envs\summary\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  # Remove the CWD from sys.path while we load stuff.


In [4]:
# FIXME: all externals at I-5 North are assigned to a single station. 
# We need to adjust these based on observed volumes between I5 and 99 and the other road...


In [5]:
# Load LEHD data
df_lehd = pd.read_csv(r'R:\e2projects_two\2023_base_year\LEHD\wa_od_main_JT00_2021.csv')
for col in ['h_geocode','w_geocode']:
    df_lehd[col] = df_lehd[col].astype('str')

In [6]:
# Get mode share based on home tract averages from ACS
import os
from census import Census
from us import states

# Note, you must obtain a census key and create a system environment variable called CENSUS_KEY that stores the key value
# You may need to completely restart anaconda/jupyter/VS to get it to recognize the new variable
c = Census(os.environ['CENSUS_KEY'])

census_year = 2022

# Total
c.acs5.state(('NAME','B08301_001E'), states.WA.fips, year=2022)

def get_acs_data(table):
    results_df = pd.DataFrame()
#     county_map = {'033': 'King',
#                    '035': 'Kitsap',
#                    '053': 'Pierce',
#                    '061': 'Snohomish',
#                   }
    df = pd.DataFrame(
        c.acs5.get(table, geo={'for': 'tract:*',
                   'in': 'state:{}'.format(states.WA.fips)})
    )
#     df['county_name'] = df['county'].map(county_map)
#     df = df[~df['county_name'].isnull()]
    df.drop('state', axis=1, inplace=True)

    return df

df_tot = get_acs_data('B08301_001E')

# B08301_003E Estimate!!Total:!!Car, truck, or van:!!Drove alone
df_sov = get_acs_data('B08301_003E')
# B08301_005E Estimate!!Total:!!Car, truck, or van:!!Carpooled:!!In 2-person carpool
df_hov2 = get_acs_data('B08301_005E')
# B08301_004E Estimate!!Total:!!Car, truck, or van:!!Carpooled:
df_hovtot = get_acs_data('B08301_004E')

# Calcualte hov3 as df_hovtot - df_hov2

In [7]:
def create_table(internal_geog, external_geog, suffix):
    
    # Join h_geocode (home block ID) to the external station. Then group by the external station to get
    # the OD table from that station to each w_geocode
    df = pd.merge(df_lehd, block_ext_gdf[['GEOID20','EXT_STA']], left_on=external_geog, right_on='GEOID20', how='left')
    df = df[~df['EXT_STA'].isnull()]

    # Get block-TAZ lookups based on internal values
    df = df.merge(block_taz_gdf[['geoid20','taz']], left_on=internal_geog, right_on='geoid20')

    df['taz'] = df['taz'].astype('int')
    df['EXT_STA'] = df['EXT_STA'].astype('int')

    df = df.groupby(['GEOID20','EXT_STA','taz']).sum()[['S000']].reset_index()
    df = df[df['EXT_STA'] != 0]
    
    df['county'] = df['GEOID20'].astype('str').apply(lambda x: x[2:5])
    df['tract'] = df['GEOID20'].astype('str').apply(lambda x: x[5:11])
    
    df = df.merge(df_tot, on=['county','tract'])
    df = df.merge(df_sov, on=['county','tract'])
    df = df.merge(df_hov2, on=['county','tract'])
    df = df.merge(df_hovtot, on=['county','tract'])
    
    df['hov3'] = df['B08301_004E']-df['B08301_005E']
    df.rename(columns={'B08301_005E': 'hov2',
                         'B08301_003E': 'sov',
                         'B08301_001E': 'acs_tot'}, 
                 inplace=True)
    
    df['SOV_Per_'+suffix] = (df['sov']/df['acs_tot'])*df['S000']
    df['HOV2_Per_'+suffix] = (df['hov2']/df['acs_tot'])*df['S000']
    df['HOV3_Per_'+suffix] = (df['hov3']/df['acs_tot'])*df['S000']

    # Convert from person trips to vehicle trips
    # Assume SOV is 1 person trip and HOV2 is 0.5 vehicles trips
    # HOV3 is 1/3.5 vehicle trips
    df['SOV_Veh_'+suffix] = df['SOV_Per_'+suffix].copy()
    df['HOV2_Veh_'+suffix] = df['HOV2_Per_'+suffix]/2
    df['HOV3_Veh_'+suffix] = df['HOV3_Per_'+suffix]/3.5

    df.rename(columns={'GEOID20': 'GEOID', 
                           'taz': 'PSRC_TAZ', 
                          'EXT_STA': 'External_Station',
                          'S000': 'Total_'+suffix},
                 inplace=True)
    
    df = df[['GEOID','PSRC_TAZ','External_Station','Total_'+suffix,
       'SOV_Per_'+suffix,'HOV2_Per_'+suffix,'HOV3_Per_'+suffix,
       'SOV_Veh_'+suffix,'HOV2_Veh_'+suffix,'HOV3_Veh_'+suffix]]
    
    # Group by TAZ and External station
    df = df.groupby(['PSRC_TAZ','External_Station']).sum().reset_index()
    
    return df

In [8]:
df_ie = create_table(internal_geog='h_geocode',external_geog='w_geocode', suffix='IE')

In [9]:
df_ei = create_table(internal_geog='w_geocode',external_geog='h_geocode', suffix='EI')

In [10]:
df_ei.head()

,PSRC_TAZ,External_Station,Total_EI,SOV_Per_EI,HOV2_Per_EI,HOV3_Per_EI,SOV_Veh_EI,HOV2_Veh_EI,HOV3_Veh_EI
0,1,3739,1,0.741712,0.061681,0.063608,0.741712,0.030840,0.018174
1,1,3740,1,0.814264,0.045319,0.031204,0.814264,0.022660,0.008915
2,1,3744,2,1.581548,0.158932,0.052241,1.581548,0.079466,0.014926
3,1,3747,1,0.538406,0.063891,0.007179,0.538406,0.031945,0.002051
4,1,3750,1,0.814463,0.135866,0.000000,0.814463,0.067933,0.000000


In [11]:
df_ie.head()

,PSRC_TAZ,External_Station,Total_IE,SOV_Per_IE,HOV2_Per_IE,HOV3_Per_IE,SOV_Veh_IE,HOV2_Veh_IE,HOV3_Veh_IE
0,1,3733,4,2.485032,0.285785,0.045332,2.485032,0.142893,0.012952
1,1,3739,2,1.341657,0.177550,0.020363,1.341657,0.088775,0.005818
2,2,3733,13,8.773226,0.997649,0.300848,8.773226,0.498825,0.085956
3,2,3739,7,5.292340,0.302589,0.063731,5.292340,0.151295,0.018209
4,2,3740,1,0.770005,0.078007,0.058379,0.770005,0.039004,0.016680


In [12]:
df = df_ei.merge(df_ie, on=['PSRC_TAZ','External_Station'])
df = df[['PSRC_TAZ','External_Station','Total_IE','Total_EI','SOV_Per_IE',
    'SOV_Per_EI','HOV2_Per_IE','HOV2_Per_EI','HOV3_Per_IE',
    'HOV3_Per_EI','SOV_Veh_IE','SOV_Veh_EI','HOV2_Veh_IE','HOV2_Veh_EI','HOV3_Veh_IE','HOV3_Veh_EI']]

In [13]:
df.to_csv(r'R:\e2projects_two\2023_base_year\externals\external_trip_distribution.csv', index=False)

In [14]:
df

,PSRC_TAZ,External_Station,Total_IE,Total_EI,SOV_Per_IE,SOV_Per_EI,HOV2_Per_IE,HOV2_Per_EI,HOV3_Per_IE,HOV3_Per_EI,SOV_Veh_IE,SOV_Veh_EI,HOV2_Veh_IE,HOV2_Veh_EI,HOV3_Veh_IE,HOV3_Veh_EI
0,1,3739,2,1,1.341657,0.741712,0.177550,0.061681,0.020363,0.063608,1.341657,0.741712,0.088775,0.030840,0.005818,0.018174
1,2,3733,13,1,8.773226,0.671972,0.997649,0.141910,0.300848,0.000000,8.773226,0.671972,0.498825,0.070955,0.085956,0.000000
2,2,3739,7,1,5.292340,0.719381,0.302589,0.057873,0.063731,0.131225,5.292340,0.719381,0.151295,0.028937,0.018209,0.037493
3,2,3740,1,2,0.770005,1.604359,0.078007,0.135237,0.058379,0.034152,0.770005,1.604359,0.039004,0.067618,0.016680,0.009758
4,2,3744,2,11,1.568731,8.166871,0.192278,0.979947,0.086888,0.316895,1.568731,8.166871,0.096139,0.489973,0.024825,0.090541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7420,3699,3748,6,2,3.810332,1.405518,0.274439,0.126327,0.193347,0.020247,3.810332,1.405518,0.137220,0.063164,0.055242,0.005785
7421,3700,3733,8,2,5.202735,1.474004,0.710328,0.159906,0.145837,0.083231,5.202735,1.474004,0.355164,0.079953,0.041668,0.023780
7422,3700,3739,1,1,0.794040,0.764902,0.088079,0.100899,0.041722,0.011988,0.794040,0.764902,0.044040,0.050450,0.011921,0.003425
7423,3700,3740,1,1,0.801858,0.870270,0.039009,0.107432,0.017957,0.003378,0.801858,0.870270,0.019505,0.053716,0.005130,0.000965
